In [ ]:
import netCDF4 as nc
import numpy as np
from scipy.interpolate import interp1d


fnumbers = [f'{num:05}' for num in range(9861, 10611+1, 30)] # last valid file 10611

for fnumber in fnumbers:
    print(fnumber)
    fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_'+str(fnumber)+'.nc'
    dataset = nc.Dataset(fname)
    angle = dataset.variables['angle']
    angle = angle[0, 0]
    u_east = dataset.variables['u_eastward']
    u_east = np.transpose(u_east, axes=(3, 2, 1, 0))
    u_east = np.flip(u_east, axis=2)
    v_north = dataset.variables['v_northward']
    v_north = np.transpose(v_north, axes=(3, 2, 1, 0))
    v_north = np.flip(v_north, axis=2)
    lon_rho  = dataset.variables['lon_rho']
    lon_rho = np.transpose(lon_rho, axes=(1, 0))
    lat_rho  = dataset.variables['lat_rho']
    lat_rho = np.transpose(lat_rho, axes=(1, 0))
    z_r = np.load('/home/z5297792/MRes/z_r.npy')
    z_r = np.transpose(z_r, (1, 2, 0))

    depths = z_r[150,150,:]

    def depth_interpolator(M,z_r,depths):

        # Dimensions of M and z_r
        dim_i, dim_j, dim_sigma, dim_time = M.shape
        assert z_r.shape == (dim_i, dim_j, dim_sigma)

        # Initialize interpolated matrix
        M_interpolated = np.zeros((dim_i, dim_j, len(depths), dim_time))

        # Interpolate for each day (time dimension)
        for t in range(dim_time):
            for i in range(dim_i):
                for j in range(dim_j):
                    # Get depth levels from z_r
                    depth_levels = z_r[i, j, :]
                    
                    # Get values of M at each depth level for current time t
                    values = M[i, j, :, t]
                    
                    # Perform interpolation
                    if np.all(np.isnan(values)):
                        # Handle case where all values are NaN (e.g., outside valid data range)
                        interpolated_values = np.full(len(depths), np.nan)
                    else:
                        # Use linear interpolation
                        interpolator = interp1d(depth_levels, values, kind='linear', bounds_error=False, fill_value=np.nan)
                        interpolated_values = interpolator(depths)
                    
                    # Assign interpolated values to M_interpolated for current time t
                    M_interpolated[i, j, :, t] = interpolated_values
            M_interpolated[:,:, 0, t] = M[:,:,0,t]

        return M_interpolated

    u_east = depth_interpolator(u_east, z_r, depths)
    v_north = depth_interpolator(v_north, z_r, depths)

    # ROTATE U_EAST AND V_NORTH TO x,y AXIS
    u = np.full_like(u_east, np.nan)
    v = np.full_like(v_north, np.nan)

    WIDTH, LENGTH, SIGMAS, TOTAL_NUM_DAYS = u.shape

    for TIME in range(30):
        u[:, :, :, TIME] = v_north[:, :, :, TIME] * np.sin(angle) + u_east[:, :, :, TIME] * np.cos(angle)
        v[:, :, :, TIME] = v_north[:, :, :, TIME] * np.cos(angle) - u_east[:, :, :, TIME] * np.sin(angle)

    del u_east, v_north

    def distance(lat1, lon1, lat2, lon2):
        # Constants
        EARTH_RADIUS = 6357000  # in meters
        # Convert latitude and longitude from degrees to radians
        lat1_rad = np.radians(lat1)
        lon1_rad = np.radians(lon1)
        lat2_rad = np.radians(lat2)
        lon2_rad = np.radians(lon2)
        # Haversine formula
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        a = np.sin(dlat / 2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        return EARTH_RADIUS * c

    # ARC LENGTHS (Distance between rho points)
    dx = distance(lat_rho[:-1, :], lon_rho[:-1, :], lat_rho[1:, :], lon_rho[1:, :])
    dy = distance(lat_rho[:, :-1], lon_rho[:, :-1], lat_rho[:, 1:], lon_rho[:, 1:])
    dx = np.repeat(dx[:, :, np.newaxis], 30, axis=2)
    dy = np.repeat(dy[:, :, np.newaxis], 30, axis=2)


    ow = np.full_like(u, np.nan)
    cur_speed = np.full_like(u, np.nan)

    for t in range(30):
        u_t = u[:,:,:,t]
        v_t = v[:,:,:,t]


        # CHANGES IN VELOCITIES
        # For u_x
        u_x = (u_t[2:,:,:] - u_t[:-2,:,:]) / (dx[:-1,:] + dx[1:,:])
        # For u_y
        u_y = (u_t[:,2:,:] - u_t[:,:-2,:]) / (dy[:,:-1] + dy[:,1:])
        # For v_x
        v_x = (v_t[2:,:,:] - v_t[:-2,:,:]) / (dx[:-1,:] + dx[1:,:])
        # For v_y
        v_y = (v_t[:,2:,:] - v_t[:,:-2,:]) / (dy[:,:-1] + dy[:,1:])

        # For u_x
        u_x = np.concatenate((np.zeros((1, LENGTH, SIGMAS)), u_x, np.zeros((1, LENGTH, SIGMAS))), axis=0)
        # For v_x
        v_x = np.concatenate((np.zeros((1, LENGTH, SIGMAS)), v_x, np.zeros((1, LENGTH, SIGMAS))), axis=0)
        # For u_y
        u_y = np.concatenate((np.zeros((WIDTH, 1, SIGMAS)), u_y, np.zeros((WIDTH, 1, SIGMAS))), axis=1)
        # For v_y
        v_y = np.concatenate((np.zeros((WIDTH, 1, SIGMAS)), v_y, np.zeros((WIDTH, 1, SIGMAS))), axis=1)
        # NORMAL STRAIN
        s_n_t = u_x - v_y
        # SHEAR STRAIN
        s_s_t = v_x + u_y
        # VORTICITY
        vort_t = v_x - u_y
        # OKUBO-WEISS
        ow_t = s_n_t**2 + s_s_t**2 - vort_t**2
        # CURRENT SPEED
        cur_speed_t = np.sqrt(u_t**2 + v_t**2)

        ow[:,:,:,t] = ow_t
        cur_speed[:,:,:,t] = cur_speed_t

        del u_x, v_x, u_y, v_y, s_n_t, s_s_t, vort_t, ow_t, cur_speed_t
    del u, v


    np.save('/srv/scratch/z5297792/Climatology/ow_and_cur_speeds/ow_'+str(fnumber)+'.npy', ow)
    np.save('/srv/scratch/z5297792/Climatology/ow_and_cur_speeds/cur_speed_'+str(fnumber)+'.npy', cur_speed)

    del ow, cur_speed